In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
# For reproducibility
torch.manual_seed(1)

이제 Softmax 함수를 사용한다:
convert numbers to probabilities w/softmax:

$$ P(class=i) = \frac{e^i}{\sum e^i} $$

In [3]:
z = torch.FloatTensor([1, 2, 3])

PyTorch에는 `softmax` 함수가 있다:

In [4]:
hypothesis = F.softmax(z, dim=0)
print(hypothesis)

tensor([0.0900, 0.2447, 0.6652])


softmax 함수의 정의상, 위 각 hypothesis 원소 값은 확률이다.

따라서 전부 더하면 1이 되어야만 한다:

In [5]:
hypothesis.sum()

tensor(1.)

## Cross Entropy Loss (Low-level)

multi-class 분류에, cross entropy loss를 사용한다.

$$ L = \frac{1}{N} \sum - y \log(\hat{y}) $$

where $\hat{y}$ is the predicted probability and $y$ is the correct probability (either 0 or 1).

In [6]:
''' softmax 함수 적용시 dim=0, 1 차이를 확인하기 위한 코드
import torch
import torch.nn.functional as F

# 입력 텐서 생성 (2x3 행렬)
z = torch.tensor([[1.0, 2.0, 3.0],
                  [4.0, 5.0, 6.0]])

# 1️⃣ dim=0 (열 방향 소프트맥스)
softmax_dim0 = F.softmax(z, dim=0)
sum_dim0 = softmax_dim0.sum(dim=0)  # 각 열의 합 확인

# 2️⃣ dim=1 (행 방향 소프트맥스)
softmax_dim1 = F.softmax(z, dim=1)
sum_dim1 = softmax_dim1.sum(dim=1)  # 각 행의 합 확인

# 출력
print("=== Softmax(dim=0) 적용 결과 ===")
print(softmax_dim0)
print("각 열의 합:", sum_dim0)

print("\n=== Softmax(dim=1) 적용 결과 ===")
print(softmax_dim1)
print("각 행의 합:", sum_dim1)
'''


' softmax 함수 적용시 dim=0, 1 차이를 확인하기 위한 코드\nimport torch\nimport torch.nn.functional as F\n\n# 입력 텐서 생성 (2x3 행렬)\nz = torch.tensor([[1.0, 2.0, 3.0],\n                  [4.0, 5.0, 6.0]])\n\n# 1️⃣ dim=0 (열 방향 소프트맥스)\nsoftmax_dim0 = F.softmax(z, dim=0)\nsum_dim0 = softmax_dim0.sum(dim=0)  # 각 열의 합 확인\n\n# 2️⃣ dim=1 (행 방향 소프트맥스)\nsoftmax_dim1 = F.softmax(z, dim=1)\nsum_dim1 = softmax_dim1.sum(dim=1)  # 각 행의 합 확인\n\n# 출력\nprint("=== Softmax(dim=0) 적용 결과 ===")\nprint(softmax_dim0)\nprint("각 열의 합:", sum_dim0)\n\nprint("\n=== Softmax(dim=1) 적용 결과 ===")\nprint(softmax_dim1)\nprint("각 행의 합:", sum_dim1)\n'

In [7]:
z = torch.rand(3, 5, requires_grad=True)    # class 5개, sample 3개
hypothesis = F.softmax(z, dim=1)    # dim=1은 두번째 dimension에 대해서 softmax를 수행한다는 뜻
print(hypothesis)

# dim=0이면 열 방향(세로)으로 softmax 적용
# dim=1이면 행 방향(가로)으로 softmax 적용->각 행의 원소 합이 1이 되는 확률 분포 생성 

# print(hypothesis.sum(dim=0)) 으로 확인해보자 dim=1로 하면 1이 안됨

tensor([[0.2645, 0.1639, 0.1855, 0.2585, 0.1277],
        [0.2430, 0.1624, 0.2322, 0.1930, 0.1694],
        [0.2226, 0.1986, 0.2326, 0.1594, 0.1868]], grad_fn=<SoftmaxBackward0>)


In [8]:
# 3개의 샘플에 대해 각각 5개 클래스 중 하나의 정답(label)을 랜덤하게 생성하는 코드

# torch.randint(high, size)는 0 이상 high 미만의 정수를 랜덤하게 생성
# (3,)은 3개의 값을 가진 1D 텐서를 만든다는 의미
# 따라서 torch.randint(5, (3,))은 0~4 사이의 정수를 랜덤하게 3개 생성

# PyTorch의 손실 함수(CrossEntropyLoss 등)는 int64(long) 타입의 
# 정답(label)을 기대하기 때문에, .long()을 사용해 변환

# 따라서 y는 각 샘플(3개)의 정답 클래스(0~4 중 하나)를 나타내는 정수 텐서
y = torch.randint(5, (3,)).long()
print(y)

tensor([0, 2, 1])


In [9]:
y_one_hot = torch.zeros_like(hypothesis)
y_one_hot.scatter_(1, y.unsqueeze(1), 1)

tensor([[1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.]])

In [10]:
cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
print(cost)

tensor(1.4689, grad_fn=<MeanBackward0>)


### Cross-entropy Loss with `torch.nn.functional`

PyTorch에는 `F.log_softmax()` 함수가 있다:

In [11]:
# low level
torch.log(F.softmax(z, dim=1))

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]], grad_fn=<LogBackward0>)

In [12]:
# high level
F.log_softmax(z, dim=1)

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]],
       grad_fn=<LogSoftmaxBackward0>)

PyTorch 에는 NLL(negative loss likelihood)을 계산하는 함수 `F.nll_loss()` 도 있다:

In [13]:
# low level
(y_one_hot * -torch.log(F.softmax(z, dim=1))).sum(dim=1).mean()

tensor(1.4689, grad_fn=<MeanBackward0>)

In [14]:
# high level
F.nll_loss(F.log_softmax(z, dim=1), y)

tensor(1.4689, grad_fn=<NllLossBackward0>)

PyTorch에는 이 둘(`F.log_softmax()` 와 `F.nll_loss()`)을 합친 `F.cross_entropy` 함수가 있다.

In [15]:
F.cross_entropy(z, y)

tensor(1.4689, grad_fn=<NllLossBackward0>)

### Training w/ low-level Cross Entropy Loss

In [16]:
x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]
y_train = [2, 2, 2, 1, 1, 1, 0, 0]
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

In [17]:
W = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

optimizer = optim.SGD([W, b], lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    hypothesis = F.softmax(x_train.matmul(W) + b, dim=1)
    y_one_hot = torch.zeros_like(hypothesis)
    y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
    cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.761050
Epoch  200/1000 Cost: 0.689991
Epoch  300/1000 Cost: 0.643229
Epoch  400/1000 Cost: 0.604117
Epoch  500/1000 Cost: 0.568256
Epoch  600/1000 Cost: 0.533922
Epoch  700/1000 Cost: 0.500291
Epoch  800/1000 Cost: 0.466908
Epoch  900/1000 Cost: 0.433507
Epoch 1000/1000 Cost: 0.399962


### training w/ `F.cross_entropy`

In [18]:
W = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

optimizer = optim.SGD([W, b], lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    z = x_train.matmul(W) + b
    cost = F.cross_entropy(z, y_train)
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.761050
Epoch  200/1000 Cost: 0.689991
Epoch  300/1000 Cost: 0.643229
Epoch  400/1000 Cost: 0.604117
Epoch  500/1000 Cost: 0.568255
Epoch  600/1000 Cost: 0.533922
Epoch  700/1000 Cost: 0.500291
Epoch  800/1000 Cost: 0.466908
Epoch  900/1000 Cost: 0.433507
Epoch 1000/1000 Cost: 0.399962


### High-level Implementation w/ `nn.Module`

In [19]:
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(4, 3) # Output이 3

    def forward(self, x):
        return self.linear(x)

In [20]:
model = SoftmaxClassifierModel()

In [21]:
optimizer = optim.SGD(model.parameters(), lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    prediction = model(x_train)
    cost = F.cross_entropy(prediction, y_train)
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 1.849513
Epoch  100/1000 Cost: 0.689894
Epoch  200/1000 Cost: 0.609259
Epoch  300/1000 Cost: 0.551218
Epoch  400/1000 Cost: 0.500141
Epoch  500/1000 Cost: 0.451947
Epoch  600/1000 Cost: 0.405051
Epoch  700/1000 Cost: 0.358733
Epoch  800/1000 Cost: 0.312912
Epoch  900/1000 Cost: 0.269522
Epoch 1000/1000 Cost: 0.241922
